In [5]:
# opening the simfin data
import pandas as pd

# Define the file path
file_path = "data/us-shareprices-daily.csv"

# Read the CSV file with semicolon separator
df = pd.read_csv(file_path, sep=';')

# Display the first few rows
print(df.head())


  Ticker  SimFinId        Date   Open   High    Low  Close  Adj. Close  \
0      A     45846  2019-04-05  81.19  81.92  81.05  81.47       78.25   
1      A     45846  2019-04-08  81.57  81.71  80.58  81.69       78.46   
2      A     45846  2019-04-09  81.56  81.72  81.27  81.42       78.20   
3      A     45846  2019-04-10  81.45  82.24  81.45  81.68       78.45   
4      A     45846  2019-04-11  81.88  81.92  80.89  81.08       77.87   

    Volume  Dividend  Shares Outstanding  
0  1502875       NaN         317515869.0  
1   783350       NaN         317515869.0  
2  1254742       NaN         317515869.0  
3   982886       NaN         317515869.0  
4  1071479       NaN         317515869.0  


In [6]:
# opening company data
# Define the file path
file_path2 = "data/us-companies.csv"

# Read the CSV file with semicolon separator
df2 = pd.read_csv(file_path2, sep=';')

# Display the first few rows
print(df.head())

  Ticker  SimFinId        Date   Open   High    Low  Close  Adj. Close  \
0      A     45846  2019-04-05  81.19  81.92  81.05  81.47       78.25   
1      A     45846  2019-04-08  81.57  81.71  80.58  81.69       78.46   
2      A     45846  2019-04-09  81.56  81.72  81.27  81.42       78.20   
3      A     45846  2019-04-10  81.45  82.24  81.45  81.68       78.45   
4      A     45846  2019-04-11  81.88  81.92  80.89  81.08       77.87   

    Volume  Dividend  Shares Outstanding  
0  1502875       NaN         317515869.0  
1   783350       NaN         317515869.0  
2  1254742       NaN         317515869.0  
3   982886       NaN         317515869.0  
4  1071479       NaN         317515869.0  


In [7]:
# Merge dfs
merged_df = pd.merge(df2, df, on="SimFinId", how="inner") 
# Display the first few rows
print(merged_df.head())

  Ticker_x  SimFinId              Company Name  IndustryId          ISIN  \
0        A     45846  AGILENT TECHNOLOGIES INC    106001.0  US00846U1016   
1        A     45846  AGILENT TECHNOLOGIES INC    106001.0  US00846U1016   
2        A     45846  AGILENT TECHNOLOGIES INC    106001.0  US00846U1016   
3        A     45846  AGILENT TECHNOLOGIES INC    106001.0  US00846U1016   
4        A     45846  AGILENT TECHNOLOGIES INC    106001.0  US00846U1016   

   End of financial year (month)  Number Employees  \
0                           10.0           16400.0   
1                           10.0           16400.0   
2                           10.0           16400.0   
3                           10.0           16400.0   
4                           10.0           16400.0   

                                    Business Summary Market        CIK  ...  \
0  Agilent Technologies Inc is engaged in life sc...     us  1090872.0  ...   
1  Agilent Technologies Inc is engaged in life sc...     us 

In [8]:
merged_df["Date"] = pd.to_datetime(merged_df["Date"], format='%Y-%m-%d')

# Transform Company Name into a string
merged_df["Company Name"] = merged_df["Company Name"].astype(str)

In [9]:
# Define the list of columns to keep
columns_to_keep = ["Ticker_x",
    "Company Name", "IndustryId", "Number Employees", "Date", "Open", 
    "High", "Low", "Close", "Adj. Close", "Volume", "Dividend", "Shares Outstanding"
]

# Select only the specified columns
merged_df = merged_df[columns_to_keep]

# Display the first few rows of the updated DataFrame
print(merged_df.head())

  Ticker_x              Company Name  IndustryId  Number Employees       Date  \
0        A  AGILENT TECHNOLOGIES INC    106001.0           16400.0 2019-04-05   
1        A  AGILENT TECHNOLOGIES INC    106001.0           16400.0 2019-04-08   
2        A  AGILENT TECHNOLOGIES INC    106001.0           16400.0 2019-04-09   
3        A  AGILENT TECHNOLOGIES INC    106001.0           16400.0 2019-04-10   
4        A  AGILENT TECHNOLOGIES INC    106001.0           16400.0 2019-04-11   

    Open   High    Low  Close  Adj. Close   Volume  Dividend  \
0  81.19  81.92  81.05  81.47       78.25  1502875       NaN   
1  81.57  81.71  80.58  81.69       78.46   783350       NaN   
2  81.56  81.72  81.27  81.42       78.20  1254742       NaN   
3  81.45  82.24  81.45  81.68       78.45   982886       NaN   
4  81.88  81.92  80.89  81.08       77.87  1071479       NaN   

   Shares Outstanding  
0         317515869.0  
1         317515869.0  
2         317515869.0  
3         317515869.0  
4       

In [10]:
# Change all Dividend missing in merged to 0 and then try to fix others
merged_df["Dividend"] = merged_df["Dividend"].fillna(0)

# Calculate the percentage of missing values per column
missing_percentage = (merged_df.isnull().sum() / len(merged_df)) * 100

# Display the missing values percentage for each column
print(missing_percentage)

Ticker_x              0.000000
Company Name          0.000000
IndustryId            0.912000
Number Employees      5.200318
Date                  0.000000
Open                  0.000000
High                  0.000000
Low                   0.000000
Close                 0.000000
Adj. Close            0.000000
Volume                0.000000
Dividend              0.000000
Shares Outstanding    8.956522
dtype: float64


In [11]:
# Fill missing IndustryId with -1 (as a placeholder)
merged_df["IndustryId"].fillna(-1, inplace=True)

# Fill Number Employees with the mean per IndustryId
merged_df["Number Employees"] = merged_df.groupby("IndustryId")["Number Employees"].transform(lambda x: x.fillna(x.mean()))

# Fill Shares Outstanding with the mean per IndustryId
merged_df["Shares Outstanding"] = merged_df.groupby("IndustryId")["Shares Outstanding"].transform(lambda x: x.fillna(x.mean()))

# Check if any missing values remain
print(merged_df.isnull().sum())

Ticker_x              0
Company Name          0
IndustryId            0
Number Employees      0
Date                  0
Open                  0
High                  0
Low                   0
Close                 0
Adj. Close            0
Volume                0
Dividend              0
Shares Outstanding    0
dtype: int64


In [12]:
magnificent_7_tickers = ["AAPL", "MSFT", "GOOGL", "GOOG", "AMZN", "NVDA", "TSLA", "META"]
# Filter dataset to only include the Magnificent 7 tickers
filtered_df = merged_df[merged_df["Ticker_x"].isin(magnificent_7_tickers)]

# Display the unique tickers in the filtered dataset (as a final check)
print("Filtered Unique Tickers:", filtered_df["Ticker_x"].unique())

# Display first few rows to confirm filtering
print(filtered_df.head())

Filtered Unique Tickers: ['AAPL' 'AMZN' 'GOOG' 'META' 'MSFT' 'NVDA' 'TSLA']
      Ticker_x Company Name  IndustryId  Number Employees       Date   Open  \
14268     AAPL    APPLE INC    101001.0          147000.0 2019-04-05  49.11   
14269     AAPL    APPLE INC    101001.0          147000.0 2019-04-08  49.10   
14270     AAPL    APPLE INC    101001.0          147000.0 2019-04-09  50.08   
14271     AAPL    APPLE INC    101001.0          147000.0 2019-04-10  49.67   
14272     AAPL    APPLE INC    101001.0          147000.0 2019-04-11  50.21   

        High    Low  Close  Adj. Close     Volume  Dividend  \
14268  49.27  48.98  49.25       47.19   74106576       0.0   
14269  50.06  49.09  50.02       47.93  103526788       0.0   
14270  50.71  49.81  49.88       47.79  143072948       0.0   
14271  50.19  49.55  50.16       48.06   86781152       0.0   
14272  50.25  49.61  49.74       47.66   83603232       0.0   

       Shares Outstanding  
14268        1.842914e+10  
14269        1

In [13]:
# Define output file path (change the path as needed)
output_path = "data/magnificent_7_filtered.csv"  # Adjust this path to your preferred location

# Save the filtered dataset to a CSV file
filtered_df.to_csv(output_path, index=False)

# Confirm save location
print(f"Filtered dataset saved to: {output_path}")

Filtered dataset saved to: data/magnificent_7_filtered.csv


In [14]:
# Create a dictionary of DataFrames, one for each company
company_dfs = {ticker: filtered_df[filtered_df["Ticker_x"] == ticker] for ticker in filtered_df["Ticker_x"].unique()}

# Display the tickers to verify
print("Companies in dataset:", company_dfs.keys())


Companies in dataset: dict_keys(['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA'])


In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Select Apple's DataFrame
apple_df = company_dfs["AAPL"]

# Drop non-numeric columns before correlation analysis
numeric_apple_df = apple_df.select_dtypes(include=[np.number])

# Compute correlation matrix
correlation_matrix = numeric_apple_df.corr()

# Display correlation matrix visually
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Apple Stock Correlation Matrix")
plt.show()

# Print correlation values sorted by Close
correlation_with_close = correlation_matrix["Close"].sort_values(ascending=False)
print(correlation_with_close)


ModuleNotFoundError: No module named 'seaborn'

In [16]:
# Define the columns to drop
columns_to_drop = [
    "IndustryId", 
    "Number Employees", 
    "Open", 
    "High", 
    "Low", 
    "Adj. Close", 
    "Dividend"
]

# Drop the columns from the general filtered_df
filtered_df = filtered_df.drop(columns=columns_to_drop)

# Display the updated dataset to verify
print(filtered_df.head())
print("Remaining columns:", filtered_df.columns)

      Ticker_x Company Name       Date  Close     Volume  Shares Outstanding
14268     AAPL    APPLE INC 2019-04-05  49.25   74106576        1.842914e+10
14269     AAPL    APPLE INC 2019-04-08  50.02  103526788        1.842914e+10
14270     AAPL    APPLE INC 2019-04-09  49.88  143072948        1.842914e+10
14271     AAPL    APPLE INC 2019-04-10  50.16   86781152        1.842914e+10
14272     AAPL    APPLE INC 2019-04-11  49.74   83603232        1.842914e+10
Remaining columns: Index(['Ticker_x', 'Company Name', 'Date', 'Close', 'Volume',
       'Shares Outstanding'],
      dtype='object')


In [17]:
# Create a dictionary of DataFrames, one for each company
company_dfs = {ticker: filtered_df[filtered_df["Ticker_x"] == ticker] for ticker in filtered_df["Ticker_x"].unique()}

# Display the tickers to verify
print("Companies in dataset:", company_dfs.keys())

Companies in dataset: dict_keys(['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA'])


In [18]:
# Select Apple's DataFrame
apple_df = company_dfs["AAPL"]

# Drop non-numeric columns before correlation analysis
numeric_apple_df = apple_df.select_dtypes(include=[np.number])

# Compute correlation matrix
correlation_matrix = numeric_apple_df.corr()

# Display correlation matrix visually
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Apple Stock Correlation Matrix")
plt.show()

# Print correlation values sorted by Close
correlation_with_close = correlation_matrix["Close"].sort_values(ascending=False)
print(correlation_with_close)

NameError: name 'np' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define the target variable (binary classification: 1 = Price Up, 0 = Price Down)
apple_df = apple_df.sort_values(by="Date")  # Ensure correct time order
apple_df["Price_Up"] = (apple_df["Close"].shift(-1) > apple_df["Close"]).astype(int)
# Calculate daily price changes
apple_df["Price_Change"] = apple_df["Close"].pct_change() * 100

# Drop last row (no next-day price available)
apple_df = apple_df.dropna()

# Select features (Close price, Volume, Shares Outstanding)
features = ["Close", "Volume", "Shares Outstanding"]
X = apple_df[features]
y = apple_df["Price_Up"]

# Split into training (80%) and testing (20%) - Keep chronological order
train_size = int(len(apple_df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Standardize the features (important for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [16]:
# Define the Logistic Regression model
log_reg = LogisticRegression()

# Set up 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(log_reg, X_train_scaled, y_train, cv=kf, scoring="accuracy")

# Train the model on the full training set
log_reg.fit(X_train_scaled, y_train)

# Predictions on the test set
y_pred = log_reg.predict(X_test_scaled)

# Evaluate Model Performance
print(f"Cross-validation Accuracy Scores: {cv_scores}")
print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f}")
print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Cross-validation Accuracy Scores: [0.52763819 0.55276382 0.53535354 0.5        0.54040404]
Mean CV Accuracy: 0.5312
Test Set Accuracy: 0.5040

Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.33      0.39       122
           1       0.51      0.67      0.58       126

    accuracy                           0.50       248
   macro avg       0.50      0.50      0.49       248
weighted avg       0.50      0.50      0.49       248



In [17]:
import pickle

# Save the trained model
with open("pkl/apple_log_reg_model.pkl", "wb") as file:
    pickle.dump(log_reg, file)

print("Model saved successfully to log_reg_model.pkl")

Model saved successfully to log_reg_model.pkl


# Try doing gradient descent to increase accuracy. 

# Next Company - AMZN


In [ ]:
# Select Apple's DataFrame
amazon_df = company_dfs["AMZN"]

# Drop non-numeric columns before correlation analysis
numeric_amazon_df = amazon_df.select_dtypes(include=[np.number])

# Compute correlation matrix
correlation_matrix = numeric_amazon_df.corr()

# Display correlation matrix visually
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Amazon Stock Correlation Matrix")
plt.show()

# Print correlation values sorted by Close
correlation_with_close = correlation_matrix["Close"].sort_values(ascending=False)
print(correlation_with_close)

In [ ]:

# Define the target variable (binary classification: 1 = Price Up, 0 = Price Down)
amazon_df = amazon_df.sort_values(by="Date")  # Ensure correct time order
amazon_df["Price_Up"] = (amazon_df["Close"].shift(-1) > amazon_df["Close"]).astype(int)
# Calculate daily price changes
amazon_df["Price_Change"] = amazon_df["Close"].pct_change() * 100

# Drop last row (no next-day price available)
amazon_df = amazon_df.dropna()

# Select features (Close price, Volume, Shares Outstanding)
features = ["Close", "Volume", "Shares Outstanding"]
X = amazon_df[features]
y = amazon_df["Price_Up"]

# Split into training (80%) and testing (20%) - Keep chronological order
train_size = int(len(amazon_df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Standardize the features (important for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define the Logistic Regression model
log_reg = LogisticRegression()

# Set up 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(log_reg, X_train_scaled, y_train, cv=kf, scoring="accuracy")

# Train the model on the full training set
log_reg.fit(X_train_scaled, y_train)

# Predictions on the test set
y_pred = log_reg.predict(X_test_scaled)

# Evaluate Model Performance
print(f"Cross-validation Accuracy Scores: {cv_scores}")
print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f}")
print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Save the trained model
with open("pkl/amazon_log_reg_model.pkl", "wb") as file:
    pickle.dump(log_reg, file)

print("Model saved successfully to log_reg_model.pkl")

# The results aren't very good, so we move on to a different kind of model. 
# Time Series!

In [18]:
"""
# from statsmodels.tsa.stattools import adfuller

# Run ADF test on Close prices
# adf_result = adfuller(apple_df["Close"])

# Print ADF results
# print("ADF Statistic:", adf_result[0])
# print("p-value:", adf_result[1])
# if adf_result[1] > 0.05:
#    print("Data is non-stationary. Differencing is required.")
# else:
#    print("Data is stationary.")
"""


'\n# from statsmodels.tsa.stattools import adfuller\n\n# Run ADF test on Close prices\n# adf_result = adfuller(apple_df["Close"])\n\n# Print ADF results\n# print("ADF Statistic:", adf_result[0])\n# print("p-value:", adf_result[1])\n# if adf_result[1] > 0.05:\n#    print("Data is non-stationary. Differencing is required.")\n# else:\n#    print("Data is stationary.")\n'

In [19]:
"""# Apply first difference if necessary
apple_df["Close_diff"] = apple_df["Close"].diff()

# Drop NaN values (since differencing creates NaNs in the first row)
apple_df = apple_df.dropna()

# Re-run ADF test on differenced data
adf_result_diff = adfuller(apple_df["Close_diff"])

# Print results
print("ADF Statistic (Differenced):", adf_result_diff[0])
print("p-value:", adf_result_diff[1])
if adf_result_diff[1] > 0.05:
    print("Differenced data is still non-stationary. Further differencing may be required.")
else:
    print("Differenced data is now stationary.")
    """


'# Apply first difference if necessary\napple_df["Close_diff"] = apple_df["Close"].diff()\n\n# Drop NaN values (since differencing creates NaNs in the first row)\napple_df = apple_df.dropna()\n\n# Re-run ADF test on differenced data\nadf_result_diff = adfuller(apple_df["Close_diff"])\n\n# Print results\nprint("ADF Statistic (Differenced):", adf_result_diff[0])\nprint("p-value:", adf_result_diff[1])\nif adf_result_diff[1] > 0.05:\n    print("Differenced data is still non-stationary. Further differencing may be required.")\nelse:\n    print("Differenced data is now stationary.")\n    '

In [20]:
"""
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ACF (Autocorrelation Function) - Checks how past values influence future values
plot_acf(apple_df["Close_diff"], lags=20)
plt.show()

# PACF (Partial Autocorrelation Function) - Identifies the direct impact of past values
plot_pacf(apple_df["Close_diff"], lags=20)
plt.show()
"""

'\nimport matplotlib.pyplot as plt\nfrom statsmodels.graphics.tsaplots import plot_acf, plot_pacf\n\n# ACF (Autocorrelation Function) - Checks how past values influence future values\nplot_acf(apple_df["Close_diff"], lags=20)\nplt.show()\n\n# PACF (Partial Autocorrelation Function) - Identifies the direct impact of past values\nplot_pacf(apple_df["Close_diff"], lags=20)\nplt.show()\n'

# Here we get to the point where we have stationary data, congratulations. 
# But it also shows that the previous day's price has little correlation with the next day's price. So we must add additional variables. 

In [21]:
"""
import talib
# Define target variable: Will the differenced price be positive?
apple_df["Price_Up"] = (apple_df["Close_diff"] > 0).astype(int)

# Compute technical indicators
apple_df["SMA_5"] = apple_df["Close_diff"].rolling(window=5).mean()
apple_df["SMA_10"] = apple_df["Close_diff"].rolling(window=10).mean()
apple_df["Volatility"] = apple_df["Close_diff"].rolling(window=5).std()
apple_df["RSI"] = talib.RSI(apple_df["Close"], timeperiod=14)
apple_df["MACD"], _, _ = talib.MACD(apple_df["Close"])

# Drop NaN values caused by rolling calculations
apple_df = apple_df.dropna()

# Compute correlation matrix
correlation_matrix = apple_df[["Price_Up", "Close_diff", "SMA_5", "SMA_10", "Volatility", "RSI", "MACD"]].corr()

# Display correlation values sorted by Price_Up
correlation_with_target = correlation_matrix["Price_Up"].sort_values(ascending=False)
print("Correlation with Price_Up:\n", correlation_with_target)

# Visualize correlation matrix
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix - Apple Stock (Fixed)")
plt.show()
"""

'\nimport talib\n# Define target variable: Will the differenced price be positive?\napple_df["Price_Up"] = (apple_df["Close_diff"] > 0).astype(int)\n\n# Compute technical indicators\napple_df["SMA_5"] = apple_df["Close_diff"].rolling(window=5).mean()\napple_df["SMA_10"] = apple_df["Close_diff"].rolling(window=10).mean()\napple_df["Volatility"] = apple_df["Close_diff"].rolling(window=5).std()\napple_df["RSI"] = talib.RSI(apple_df["Close"], timeperiod=14)\napple_df["MACD"], _, _ = talib.MACD(apple_df["Close"])\n\n# Drop NaN values caused by rolling calculations\napple_df = apple_df.dropna()\n\n# Compute correlation matrix\ncorrelation_matrix = apple_df[["Price_Up", "Close_diff", "SMA_5", "SMA_10", "Volatility", "RSI", "MACD"]].corr()\n\n# Display correlation values sorted by Price_Up\ncorrelation_with_target = correlation_matrix["Price_Up"].sort_values(ascending=False)\nprint("Correlation with Price_Up:\n", correlation_with_target)\n\n# Visualize correlation matrix\nplt.figure(figsize=(1

In [22]:
"""
# Drop weakly correlated features
features_to_drop = [ "Volatility", "MACD"]
apple_df = apple_df.drop(columns=features_to_drop)

# Verify remaining features
print("Remaining features:", apple_df.columns)
"""

'\n# Drop weakly correlated features\nfeatures_to_drop = [ "Volatility", "MACD"]\napple_df = apple_df.drop(columns=features_to_drop)\n\n# Verify remaining features\nprint("Remaining features:", apple_df.columns)\n'

In [23]:
"""
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define features and target variable
features = ["SMA_5", "SMA_10", "RSI"]
X = apple_df[features]
y = apple_df["Price_Up"]

# Split into training (80%) and testing (20%) - Keep chronological order
train_size = int(len(apple_df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Logistic Regression model
log_reg = LogisticRegression()

# Set up 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(log_reg, X_train_scaled, y_train, cv=kf, scoring="accuracy")

# Train the model on full training set
log_reg.fit(X_train_scaled, y_train)

# Predictions on the test set
y_pred = log_reg.predict(X_test_scaled)

# Evaluate Model Performance
print(f"Cross-validation Accuracy Scores: {cv_scores}")
print(f"Mean CV Accuracy: {np.mean(cv_scores):.4f}")
print(f"Test Set Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
"""


'\nimport numpy as np\nfrom sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import accuracy_score, classification_report\n\n# Define features and target variable\nfeatures = ["SMA_5", "SMA_10", "RSI"]\nX = apple_df[features]\ny = apple_df["Price_Up"]\n\n# Split into training (80%) and testing (20%) - Keep chronological order\ntrain_size = int(len(apple_df) * 0.8)\nX_train, X_test = X.iloc[:train_size], X.iloc[train_size:]\ny_train, y_test = y.iloc[:train_size], y.iloc[train_size:]\n\n# Standardize the features\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n\n# Define Logistic Regression model\nlog_reg = LogisticRegression()\n\n# Set up 5-fold cross-validation\nkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n\n# Perform cross-validation\ncv_sco

In [24]:
"""
# Ensure DataFrame has a proper integer index
apple_df = apple_df.reset_index(drop=True)

# Initialize Signal column as Hold
apple_df["Signal"] = "Hold"  

holding = False  # Tracks whether we currently own the stock

# Iterate through each row except the last
for i in range(len(apple_df) - 1):  
    if pd.notna(apple_df.loc[i, "Close"]):  # Use .loc[] instead of .at[]
        if apple_df.loc[i, "Price_Up"] == 1:  # Price predicted to go up
            if not holding:  
                apple_df.loc[i, "Signal"] = "Buy"
                holding = True  # We now own the stock
            else:
                apple_df.loc[i, "Signal"] = "Hold"
        else:  # Price predicted to go down
            if holding:
                apple_df.loc[i, "Signal"] = "Sell"
                holding = False  # We no longer own the stock
            else:
                apple_df.loc[i, "Signal"] = "Hold"

# Display sample of updated Signal column
print(apple_df[["Date", "Close", "Price_Up", "Signal"]].tail(20))
"""


'\n# Ensure DataFrame has a proper integer index\napple_df = apple_df.reset_index(drop=True)\n\n# Initialize Signal column as Hold\napple_df["Signal"] = "Hold"  \n\nholding = False  # Tracks whether we currently own the stock\n\n# Iterate through each row except the last\nfor i in range(len(apple_df) - 1):  \n    if pd.notna(apple_df.loc[i, "Close"]):  # Use .loc[] instead of .at[]\n        if apple_df.loc[i, "Price_Up"] == 1:  # Price predicted to go up\n            if not holding:  \n                apple_df.loc[i, "Signal"] = "Buy"\n                holding = True  # We now own the stock\n            else:\n                apple_df.loc[i, "Signal"] = "Hold"\n        else:  # Price predicted to go down\n            if holding:\n                apple_df.loc[i, "Signal"] = "Sell"\n                holding = False  # We no longer own the stock\n            else:\n                apple_df.loc[i, "Signal"] = "Hold"\n\n# Display sample of updated Signal column\nprint(apple_df[["Date", "Close

In [25]:
"""
# Define the file path (update the path as needed)
output_path = "data/apple_stock_predictions.csv"

# Save the DataFrame as a CSV file
apple_df.to_csv(output_path, index=False)

# Confirm save location
print(f"Apple stock predictions saved to: {output_path}")
"""


'\n# Define the file path (update the path as needed)\noutput_path = "data/apple_stock_predictions.csv"\n\n# Save the DataFrame as a CSV file\napple_df.to_csv(output_path, index=False)\n\n# Confirm save location\nprint(f"Apple stock predictions saved to: {output_path}")\n'